# TOPOPLOT

Script para el cálculo y visualización del topoplot de un paciente a partir de un archivo .edf

## Resumen

En este script se realizan los cálculos de la potencia de la señal, la ubicación de los electrodos de la medición según el modelo New York, para finalmente realizar el topoplot por medio de la biblioteca "MNE" [1]

## Entradas

Por medio de argparse se definen las siguientes entradas.

1) -i o --archive: nombre del archivo .edf que desea utilizar, es de tipo str, ejemplo: -i sujeto_base.edf

2) -o o --figure: nombre y dirección para guardar el topoplot, ejemplo -o pictures/figure

3) -c o --config: nombre del archivo de configuración, éste debe incluir las siguientes dos variables: 
3.1) labels: de tipo lista, incluye el nombre de los electrodos según el modelo que se desea utilizar, por default se utiliza el modelo New York.
3.2) elec: array con las coordenadas de los electrodos según el modelo que se desea utilizar, por default se utiliza el modelo New York. sus dimensiones son el número de electrodos x 2. Debe ser de la misma longitud de la cantidad de elementos de "labels"

Ejemplo: -c config


## Salidas

1) fig: archivo .ply o imagen con el topoplot para el archivo .edf

## Uso

Como una breve descripción del código se puede decir que se comienza con la importación de la clase en donde se realizan todos los procesos y después se realiza el paso a paso por todos los metodos.

In [ ]:
from topoplot import edf_topoplot
ob = edf_topoplot()
archi,output,config = ob.argparse()
# prepara las etiquetas y coordenas del modelo de referencia del archivo config
elec,labels_ref,cz,radius = ob.preparate_ref(config)
# da lectura al edf
signal,nch,channels_labels = ob.read_edf(archi)
# calculo de la potencia de la senal
pot_signal = ob.calc_power(signal)
# calculo de las posiciones de los electrodos de la medida
pos,pot_signal,channels_labels= ob.calc_positions(nch,channels_labels,elec,labels_ref,pot_signal)
# interpolación con los electrodos de referencia
pot_interp = ob.interpolation(pot_signal,pos,elec)
# elaboración del borde circular por medio de nodos 
elec_with_circle = ob.circle(elec,cz,radius)
# interpolación con los nodos del borde
pot_interp_with_circle = ob.interpolation(pot_interp,elec,elec_with_circle)
# topoplot utilizando biblioteca mne
fig = ob.topoplot(pot_interp,elec,labels_ref)
# elaboración de malla y faces
faces = ob.mesh(elec_with_circle)
# jet to RGB
colorVal = ob.RGB(pot_interp_with_circle)
# escritura del archivo .ply
file = ob.write_ply(output,elec_with_circle,faces,colorVal)

# ejemplo
# python main_topo.py -i sujeto_base.edf -o fig3 -c config

## Bibliotecas utilizadas

[1] MNE        http://www.martinos.org/mne/

[2] pyedflib   http://pyedflib.readthedocs.io

[3] Numpy      http://www.numpy.org/

[4] Argparse   https://docs.python.org/3/library/argparse.html

[5] Scipy      http://www.scipy.org

[6] Matplotlib http://www.matplotlib.org